# PyMem3DG Tutorial 2 - Ensure Mesh Quality
`Cuncheng Zhu, Christopher T. Lee`

This tutorial demonstrate the complementary functionalities of PyMem3DG to ensure the quality of mesh. The tutorial does not mean to be extensive but to provide the flavor and methods that PyMem3DG uses. The extensive documentations is hosted at https://rangamanilabucsd.github.io/Mem3DG/.

To demonstrate, we set up the system to solve the remaining problem from Tutorial 1, evolving closed spherical membrane with high curvature deformation. Again, the following integration is pre-runned. Uncomment $\texttt{fe.integrate()}$ to rerun them.

In [5]:
import pymem3dg as dg

icoFace, icoVertex = dg.getIcosphere(1, 3)

o = dg.Options()
p = dg.Parameters()
### bending ###
p.Kbc = 8.22e-5 
### surface tension ###
p.Ksg = 0.1
### osmotic force ###
p.Kv = 0.05
p.cam = 0.40

The setup is exactly the same as the last example in tutorial 1. Without any mesh curation, the mesh will be ill-formed as the following snapshot.

<img src="output/tutorial1/star/final.png" width="400" height="200">

## Mesh regularization
The first strategy is to regularize the mesh, restricting the tangential movement of vertices. Regularization does not change the mesh connectivity and the number of elements in the mesh, which leads to benefit of efficiency and ease of output. The three type of regularization provided by PyMem3DG includes constraints on edge length, local triangle area, and local corner angles, adjusted by $K_{se}$, $K_{sl}$ and $K_{st}$, respectively. 

In [6]:
p.Kst = 1e-7
p.Ksl = 1e-7
p.Kse = 1e-7
g = dg.System(icoFace, icoVertex, p, o)
fe = dg.Euler(f = g, dt = 0.1, total_time = 10000, tSave = 1000, tolerance = 3e-5, outputDir = "output/tutorial2/ico_reg")
fe.verbosity = 5
# fe.integrate()

With the above regularization, the mesh at the same time ($T = 10000$) looks significantly nicer, with all triangles closed to equilateral. 

<img src="output/tutorial2/ico_reg/final.png" width="400" height="200">

### Reference mesh
We could also input a reference mesh that has better mesh quality than the initial mesh. For example, we could continue the above simulation from $T = 5000$ to $T=15000$ and have the reference mesh still being the icosphere.

In [8]:
inputMesh = "output/tutorial2/ico_reg/frame5.ply"
refMesh =  "output/tutorial2/ico_reg/frame0.ply"
g = dg.System(inputMesh, refMesh, p, o)
fe = dg.Euler(f = g, dt = 0.1, total_time = 10000, tSave = 1000, tolerance = 3e-5, outputDir = "output/tutorial2/bad_reg")
fe.verbosity = 5
# fe.integrate()"

The results is 

<img src="output/tutorial2/ico_reg/final.png" width="400" height="200">

Free free to adjust the values of each type of regularization and attain intuition on the behavior. 

As you might observe, there are several disadvantages using the method of regularization. The behavior of simulation will depend on the strength of regularization. Local angle penalty is less restrictive and will minimally affect the underlying physics, but the resolution on high curvature region becomes very coarse. We could combine it with penalty on local area and more strongly edge length, but its influence on physics and optimization can not be neglected. In summary, how to find good balance between restriction and flexibility is not obvious. 

## Vertex shift
In addition, instead of constantly applying regularization force, one could also regularize the mesh once in a while, by "centering" the vertices (*barycenter* to be exact). This can be done by toggling the vertex shift option as follows,


In [10]:
p.Kst = 0
p.Ksl = 0
p.Kse = 0
o.isVertexShift = True

g = dg.System(icoFace, icoVertex, p, o)
fe = dg.Euler(f = g, dt = 0.1, total_time = 10000, tSave = 1000, tolerance = 3e-5, outputDir = "output/tutorial2/ico_shift")
fe.verbosity = 5
# fe.integrate()

which also lead to a good mesh

<img src="output/tutorial2/ico_shift/final.png" width="400" height="200">

## Mesh mutation

To resolve the challenges of regularization, PyMem3DG supports adaptive mesh by mesh mutation, including edge flip, edge split and edge collapse. Because of challenges mentioned above, mesh mutation should always be turned on when running complex simulation with large deformation. 

Notice that mesh mutation will most likely increase the computational cost and size of output files. 

The computational cost involves the time to loop through elements and decide whether conduct mesh mutation, and the subsequent overheads needed for recomputation of cached quantities. Such operation happens in the frequency of data output, therefore increasing the number of data output could better prevent the deterioration of mesh, but will increase computation.

Because of the varying mesh sizes, instead of single trajectory file using high performance *NetCDF* file, output file consists of series of snapshots in $\texttt{.ply}$ format. The detail of output files and visualization will be covered in the other tutorial. 

In [12]:
p.Kst = 0
p.Ksl = 0
p.Kse = 0

o.isEdgeFlip = True
o.isSplitEdge = True
o.isCollapseEdge = True
o.isVertexShift = True

g = dg.System(icoFace, icoVertex, p, o)

g.meshMutator.flipNonDelaunay = True
g.meshMutator.splitLarge = True
g.meshMutator.splitFat = True
g.meshMutator.splitSkinnyDelaunay = True
g.meshMutator.splitCurved = True
g.meshMutator.curvTol = 0.005
g.meshMutator.collapseSkinny = True

fe = dg.Euler(f = g, dt = 1, total_time = 40000, tSave = 1000, tolerance = 3e-5, outputDir = "output/tutorial2/ico_mut")
fe.verbosity = 5
# fe.integrate()

$\texttt{meshMutator}$ is used to specify conditions for mesh mutation, which should be specified after the instantiation of $\texttt{System}$. For details, please refer to the documentation. With mutation enabled, snapshots of the shape at $T = 10000, 20000, 25000, 30000, 40000$ are the following. Mesh remains well-conditioned and well-resoluted during the simulation.

<img src="output/tutorial2/ico_mut/screenshot_frame000010.png" width="400" height="200">
<img src="output/tutorial2/ico_mut/screenshot_frame000020.png" width="400" height="200">
<img src="output/tutorial2/ico_mut/screenshot_frame000025.png" width="400" height="200">
<img src="output/tutorial2/ico_mut/screenshot_frame000030.png" width="400" height="200">
<img src="output/tutorial2/ico_mut/screenshot_frame000040.png" width="400" height="200">

## Additional notes: 
 
### Reference mesh when mutation
Similarly, in theory we could also support specifying additional reference mesh. However, it is only necessary if the current input mesh has large deviation in total surface area from the reference mesh. Since membrane is nearly unstretchable, normally total surface area remains and self-referencing is sufficient. At the time of writing the tutorial, PyMem3DG will throw run-time error when the topology of reference mesh does not agree with the input mesh.

### Regularization + mutation
PyMem3DG does not recommend using regularization in conjunction with mesh mutation because it is most likely unnecessary to do so (as we see in previous example, vertex shift somewhat fills the role of regularization) and the behavior is not fully tested.